In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import seaborn as sns
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url,root='.')

100%|██████████| 135107811/135107811 [00:09<00:00, 13859388.18it/s]


In [4]:
with tarfile.open('/kaggle/working/cifar10.tgz') as trf:
    trf.extractall('./data')

In [5]:
classes = dict(enumerate(os.listdir('/kaggle/working/data/cifar10/train')))

In [6]:
classes

{0: 'frog',
 1: 'dog',
 2: 'airplane',
 3: 'automobile',
 4: 'ship',
 5: 'deer',
 6: 'truck',
 7: 'bird',
 8: 'horse',
 9: 'cat'}

In [7]:
img = os.listdir('/kaggle/working/data/cifar10/train/frog')[0]

In [8]:
from PIL import Image
img_ = Image.open('/kaggle/working/data/cifar10/train/frog/'+img)

In [9]:
img_.size

(32, 32)

In [10]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms,ToTensor,RandomRotation

In [20]:
transform = transforms.Compose((
    RandomRotation(13),
    ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
)

In [12]:
train_dataset = ImageFolder('/kaggle/working/data/cifar10/train',transform=transform)
test_dataset = ImageFolder('/kaggle/working/data/cifar10/test',transform=transform)

In [15]:
type(train_dataset)

torchvision.datasets.folder.ImageFolder

In [13]:
train_dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [14]:
test_dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [16]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset = train_dataset,batch_size=16,shuffle=True)
test_dataloader = DataLoader(dataset = test_dataset,batch_size=16,shuffle=True)

In [18]:
count =0
for image,label in train_dataloader:
    if(count==10): 
        break
    print(label)
    count+=1

tensor([6, 7, 1, 9, 1, 2, 2, 7, 7, 2, 4, 6, 9, 3, 0, 5])
tensor([6, 0, 4, 5, 8, 1, 6, 0, 3, 3, 7, 4, 9, 3, 6, 2])
tensor([9, 6, 5, 2, 9, 1, 6, 8, 8, 2, 4, 0, 9, 8, 6, 8])
tensor([6, 0, 5, 6, 6, 2, 1, 1, 7, 3, 9, 9, 0, 7, 8, 9])
tensor([4, 9, 7, 0, 2, 6, 0, 0, 4, 6, 0, 9, 5, 9, 0, 9])
tensor([0, 9, 3, 2, 5, 8, 5, 4, 1, 8, 9, 7, 1, 5, 9, 7])
tensor([5, 3, 1, 7, 0, 9, 6, 9, 9, 1, 9, 5, 8, 9, 2, 3])
tensor([1, 1, 4, 8, 5, 0, 0, 7, 2, 2, 1, 0, 4, 4, 9, 3])
tensor([8, 9, 8, 6, 8, 9, 2, 9, 5, 3, 7, 9, 8, 8, 0, 0])
tensor([9, 1, 2, 4, 7, 0, 2, 6, 9, 2, 9, 0, 6, 3, 7, 9])


In [19]:
from torch import nn

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,activation_function):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.activation = activation_function
        self.kernel_size = kernel_size
        self.block = nn.ModuleList([
            nn.Conv2d(in_channels = self.in_channels,
                      out_channels= self.out_channels,
                      padding=1,
                      kernel_size=self.kernel_size,
                      stride=1,
                      bias=False),
            self.activation(True),
            nn.BatchNorm2d(self.out_channels),
            nn.Conv2d(in_channels = self.out_channels,
                      out_channels= self.out_channels,
                      kernel_size=1,
                      stride=1,
                      bias=False),
            self.activation(True),
            nn.BatchNorm2d(self.out_channels),
        ])
        
    def forward(self,X):
        out = X
        for layer in self.block:
            out = layer(out)
        print(out.shape)
        print(X.shape)
        return X +out

In [ ]:
class ResNet(nn.Module):
    def __init__(self,in_channels, num_classes):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.filters = 64
        
        self.con1 = nn.Conv2d(in_channels=3,
                             out_channels=self.filters**1,
                             kernel_size=7,
                             stride=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.block1 = nn.Sequential(
                        ResNetBlock(self.filters**1,self.filters**1,kernel_size=3,activation_function=nn.ReLU),
                        ResNetBlock(self.filters**1,self.filters**1,kernel_size=3,activation_function=nn.ReLU),
        )
        
        self.conv2 = nn.Conv2d(in_channels=self.filters**1,out_channels=self.filters**2,kernel_size=3,stride=2)        
        self.block2 = nn.Sequential(
                        ResNetBlock(self.filters**2,self.filters**2,kernel_size=3,activation_function=nn.ReLU),
                        ResNetBlock(self.filters**2,self.filters**2,kernel_size=3,activation_function=nn.ReLU),
        )
        
        self.conv3 = nn.Conv2d(in_channels=self.filters**2,out_channels=self.filters**3,kernel_size=3,stride=2)        
        self.block3 = nn.Sequential(
                        ResNetBlock(self.filters**3,self.filters**3,kernel_size=3,activation_function=nn.ReLU),
                        ResNetBlock(self.filters**3,self.filters**3,kernel_size=3,activation_function=nn.ReLU),
        )
    def forward(x):
        x = self.co1(x)
        x = self.maxpool1(x)
        x = self.block1(x)
        x = self.conv2(x)
        x = self.block2(x)
        x = self.conv3(x)
        x = self.block3(x)
        print(x.shape)
        
        return x

In [ ]:
model = ResNet(3,10).to('cuda')
model

In [ ]:
model(random_sample).shape

In [15]:
from torch import nn
class ResNetPair(nn.Module):

  def __init__(self,in_channels=3,out_channels=3,kernel_size=3,activation_function=nn.ReLU(),padding=1,downsample = True):
    super(ResNetPair,self).__init__()
    self.downsample           = downsample
    self.activation_function  = activation_function
    self.in_channels          = in_channels
    self.out_channels         = out_channels
    self.kernel_size          = kernel_size
    self.padding              = padding
    layers = []
    self.shortcut             = None

    if downsample:
      layers.append(
          nn.Conv2d(in_channels = self.in_channels,out_channels=self.out_channels,stride=2,kernel_size=self.kernel_size,padding = self.padding)
      )
      self.shortcut =  nn.Conv2d(in_channels = self.in_channels,out_channels=self.out_channels,stride=2,kernel_size=self.kernel_size,padding = self.padding)
    else:
      layers.append(
          nn.Conv2d(in_channels = self.in_channels,out_channels=self.out_channels,stride=1,kernel_size=self.kernel_size,padding = self.padding)
      )
      self.shortcut =  nn.Conv2d(in_channels = self.in_channels,out_channels=self.out_channels,stride=1,kernel_size=self.kernel_size,padding = self.padding)


    layers.append(
        nn.BatchNorm2d(self.out_channels,)
    )
    layers.append(
        self.activation_function
    )
    layers.append(
        nn.Conv2d(in_channels = self.out_channels,out_channels=self.out_channels,stride=1,kernel_size=self.kernel_size,padding = self.padding)
    )
    layers.append(
        nn.BatchNorm2d(self.out_channels)
    )
    layers.append(
        self.activation_function
    )

    self.ResNetPair = nn.Sequential(*layers)

  def forward(self,X):
      residual  = X
      out       = self.ResNetPair(X)
      out      += self.shortcut(residual)
      return out


class ResNetBlock(nn.Module):
    def __init__(self,in_channels=3, out_channels=3, kernel_size=3, num_layers_per_block = 4, padding = 1,activation_function = nn.ReLU(),downsample = True):
      super(ResNetBlock,self).__init__()
      self.padding              = padding
      self.downsample           = downsample
      self.kernel_size          = kernel_size
      self.in_channels          = in_channels
      self.out_channels         = out_channels
      self.activation_function  = activation_function
      self.num_layers_per_block = num_layers_per_block

      layers = []
      layers.append(
          ResNetPair(in_channels=self.in_channels,out_channels=self.out_channels,kernel_size=self.kernel_size,activation_function=self.activation_function,padding=self.padding,downsample=self.downsample)
      )
      for block in range(1,self.num_layers_per_block):
          layers.append(
          ResNetPair(in_channels=self.out_channels,out_channels=self.out_channels,kernel_size=self.kernel_size,activation_function=self.activation_function,padding=self.padding,downsample=False)
      )

      self.ResNetBlock = nn.Sequential(*layers)
    def forward(self,X):
      out = self.ResNetBlock(X)
      return out

class ResNet16(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet16, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64)
        self.layer2 = self._make_layer(64, 128, downsample=True)
        self.layer3 = self._make_layer(128, 256, downsample=True)
        self.layer4 = self._make_layer(256, 512, downsample=True)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, self.num_classes)
    def _make_layer(self, in_channels, out_channels, downsample=False):
        return(ResNetBlock(in_channels, out_channels, downsample=downsample,num_layers_per_block=2))

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        print(x.shape)
        x = x.permute(0,2,3,1)
        x = x.view(x.size(0),x.size(1)*x.size(2)*x.size(3))
        x = self.fc(x)
        return x

# Create an instance of ResNet16
resnet_18 = ResNet16(num_classes=37)

# Print the model architecture
(resnet_18)

ResNet16(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): ResNetBlock(
    (activation_function): ReLU()
    (ResNetBlock): Sequential(
      (0): ResNetPair(
        (activation_function): ReLU()
        (shortcut): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (ResNetPair): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU()
        )
      )
      (1): ResNetPair(
        (act

In [35]:
ans = resnet_18(torch.randn(1,3,128,128))

torch.Size([1, 512, 1, 1])


In [36]:
ans.shape

torch.Size([1, 37])

In [ ]:
ans

In [31]:
true = torch.randint(0,37,(1,))

In [19]:
assert (true.shape==ans.shape)

AssertionError: 

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
# Training cylce
model = model.to('cuda')
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

N_EPOCH = 50

for epoch in range(N_EPOCH):
    val_loss = 0. 
    
    
    for image,label in train_dataloader:
        image = image.to('cuda')
        label = label.to('cuda')
        
        pred = model(image)
        train_loss = loss(pred,label)
        
        
